In [1]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import Settings
import os

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"



Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

In [2]:

!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'



--2024-04-23 21:31:04--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: 'data/10q/uber_10q_march_2022.pdf'

data/10q/uber_10q_m 100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2024-04-23 21:31:05 (20.8 MB/s) - 'data/10q/uber_10q_march_2022.pdf' saved [1260185/1260185]

--2024-04-23 21:31:05--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercon

In [3]:
# Load Data
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()



In [4]:
# Build Indices
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)

# Build Query Engine
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

# Build List of query engines
query_engine_tools = [
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(
            name="sept_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " September 2022"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(
            name="june_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " June 2022"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(
            name="march_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " March 2022"
            ),
        ),
    ),
]

# Buld Final Chained Query engine
s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

In [10]:
import nest_asyncio

nest_asyncio.apply()
response = s_engine.query(
    "Analyze Uber revenue growth over the latest two quarter filings over the same period last year"
)
print(response)

Generated 2 sub questions.
[sept_22] Q: What was Uber's revenue in the latest quarter ending September 2022?
[june_22] Q: What was Uber's revenue in the previous quarter ending June 2022?
[sept_22] A: Uber's revenue in the latest quarter ending September 2022 was $8.3 billion.
[june_22] A: Uber's revenue in the previous quarter ending June 2022 was $8.1 billion.
Uber's revenue grew from $8.1 billion in the previous quarter ending June 2022 to $8.3 billion in the latest quarter ending September 2022. Comparing these figures to the same period last year, there is an increase in revenue over the latest two quarter filings.


In [7]:
print(response)

Uber's revenue increased from $8,073 million in the quarter ending June 2022 to $8.3 billion in the latest quarter ending September 2022, indicating a growth in revenue over the two quarters.


In [11]:
response = s_engine.query(
    "Analyze change in macro environment over the 3 quarters"
)
print(response)

Generated 3 sub questions.
[sept_22] Q: What were the financial results for Uber in September 2022?
[june_22] Q: What were the financial results for Uber in June 2022?
[march_22] Q: What were the financial results for Uber in March 2022?
[june_22] A: In June 2022, Uber reported a significant increase in revenue compared to the same period in 2021. The revenue for June 2022 was $8,073 million, representing a 105% increase from June 2021. Additionally, Uber experienced improvements in its operating performance, with a decrease in loss from operations compared to the previous year.
[march_22] A: In March 2022, Uber reported revenue of $6.9 billion, representing a 136% increase compared to the same period in 2021. The net loss attributable to Uber Technologies, Inc. was $5.9 billion, which included various unrealized losses on debt and equity securities. Adjusted EBITDA was $168 million, showing an increase of $527 million compared to the same period in 2021. The company ended the quarter 

In [12]:
response = s_engine.query("How much cash did Uber have in sept 2022")

Generated 1 sub questions.
[sept_22] Q: What was the cash position of Uber in September 2022?
[sept_22] A: Uber had $4.9 billion in cash and cash equivalents in September 2022, with $2.9 billion held by foreign subsidiaries and the remaining balance held in the United States.
